#### Extracción tabla datos Selenium

In [1]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')
from selenium.webdriver.common.by import By
from IPython.display import Image # Para mostrar imágenes en python
from bs4 import BeautifulSoup as bs
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

pd.set_option('display.max_columns', None)

In [2]:
from selenium import webdriver
PATH = 'driver/chromedriver.exe'
driver = webdriver.Chrome()

Cache folder (C:\Users\jacom\.cache\selenium) cannot be created: No se puede crear un archivo que ya existe. (os error 183)
Cache folder (C:\Users\jacom\.cache\selenium) cannot be created: No se puede crear un archivo que ya existe. (os error 183)


In [3]:
url = 'https://www.portsdebalears.com/es/buques-en-puerto' # Definimos url de búsqueda

In [4]:
driver.get(url)

In [5]:
driver.find_element(By.XPATH, '//*[@id="popup-buttons"]/button[1]').click() # Clickamos en cookies

In [6]:
# Ojo, la tabla que queremos extraer está dentro de un frame distinto al de la página principal (por eso la url no cambia al buscar tablas de datos diferentes)

driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="giswebframe"]'))

In [7]:
driver.find_element(By.ID, 'trafico_menu').click() # Ahora si podemos clickar sobre los datos históricos

In [8]:
# Me defino una función con 

actions = ActionChains(driver)
def triple_click(element_x):
    actions.click(element_x).click(element_x).click(element_x).perform()

In [ ]:
/html/body/div[4]/div[2]/div[3]/div/div[1]/div[2]/input
/html/body/div[4]/div[2]/div[3]/div/div[1]/div[4]/input

In [9]:
# Nuestro próximo reto es introducir las fechas que queremos de rango
# Tendremos que clickar en la caja y, mejor que clickar sobre fecha, introducir por texto el rango de fechas que nos interesa

fecha_ini = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[3]/div/div[1]/div[2]/input') # Apunto con el full XPATH
triple_click(fecha_ini)
fecha_ini.send_keys("01/08/2023") # Meto fecha entrada
time.sleep(2)

fecha_fin = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[3]/div/div[1]/div[4]/input') # Apunto con el full XPATH
triple_click(fecha_fin)
fecha_fin.send_keys("31/08/2023") # Meto fecha salida
time.sleep(2)
driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[3]/div/div[1]/div[5]/button[1]/span').click()
time.sleep(10)

In [ ]:
/html/body/div[4]/div[8]/div/div[2]/div/div/div/div[2]/div[3]/div[3]/div/table/tbody

Habiendo introducido las fechas vamos a obtener la tabla de datos de transito de buques

In [10]:
time.sleep(5)
tabla = driver.find_element(By.XPATH, '/html/body/div[4]/div[8]/div/div[2]/div/div/div/div[2]/div[3]/div[3]/div/table/tbody') # Asignamos variable tabla al cuerpo de la tabla
filas = tabla.find_elements(By.TAG_NAME, 'tr') # Cada fila está metida en un 'tr'

data = []

for f in filas:

    elementos = f.find_elements(By.TAG_NAME, 'td') # Cada fila está metida en un 'td'

    tmp = []

    for e in elementos:
        tmp.append(e.text)

    data.append(tmp)

In [11]:
df_transito = pd.DataFrame(data)

In [12]:
df_transito

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,,,,,,,,,ELEANOR ROOSEVELT,,,IBIZA,IBIZA,31/07/2023 22:36,01/08/2023 07:49,,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2772,CHIPRE,123,4,,,,,,,,,,,,,,
2,,,,,,,,,,MARELLA DISCOVERY 2,,,Valencia,PALERMO,01/08/2023 04:12,01/08/2023 22:35,,ALIN.NORTE DE LA PLATAFORMA (del 24 al 2),INTERCRUISES SHORESIDE & PORT SERVICES,69.472,559,BAHAMAS,264,8,,,,,,,,,,,,,,
3,,,,,,,,,,GNV SPIRIT,,,BARCELONA,BARCELONA,01/08/2023 04:51,01/08/2023 10:33,,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,32.728,3174,ITALIA,204,6,,,,,,,,,,,,,,
4,,,,,,,,,,HEDY LAMARR,,,Valencia,Valencia,01/08/2023 05:02,01/08/2023 10:15,,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2913,CHIPRE,186,7,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,,,,,,,,,,CIUDAD DE ALCUDIA,,,Valencia,Valencia,31/08/2023 06:57,31/08/2023 11:25,,1ER. TRAMO EXT MLLES CIALES. (del 12 al 2),"GRIMALDI LOGISTICA ESPAÑA, SL",33.588,3398,ESPAÑA,183,7,,,,,,,,,,,,,,
338,,,,,,,,,,YACHT SERVANT,,,GENOA,PORT EVERGLADES,31/08/2023 08:57,02/09/2023 16:05,,M. 2ª ALIN. PONIENTE SUR (del 18 al 2),MIGUEL PUIGSERVER S A,44.522,3408,PAÍSES BAJOS,214,5,,,,,,,,,,,,,,
339,,,,,,,,,,ABSA UNO,,,SAN PEDRO DEL PINATAR,ALMERIA,31/08/2023 10:30,31/08/2023 18:50,,MUELLE ADOSADO (del 16 al 12),ESTELA SHIPPING PALMA SA,317,3353,ESPAÑA,37,5,,,,,,,,,,,,,,
340,,,,,,,,,,ECO ADRIATICA,,,CAGLIARI,SAGUNTO,31/08/2023 13:30,31/08/2023 15:28,,1ER. TRAMO EXT MLLES CIALES. (del 15 al 2),"GRIMALDI LOGISTICA ESPAÑA, SL",67.311,3338,ITALIA,238,7,,,,,,,,,,,,,,


In [ ]:
driver.quit()

In [ ]:
driver.quit()